In [ ]:
from google.colab import drive
drive.mount("/content/drive/", force_remount=True)

In [ ]:
cd /content/drive/MyDrive/

Download the pytorch-lightning/wandb packages

In [ ]:
!pip install git+https://github.com/PyTorchLightning/pytorch-lightning

In [ ]:
!pip install wandb

In [ ]:
import wandb
wandb.login(relogin=True)
wandb.login()
# wandb.__version__

Change the directory to where the source was uploaded

In [ ]:
cd /content/drive/MyDrive/Project/step2-3

Import some underlying functions

In [ ]:
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader

import time
import importlib
import warnings

Specify the path of Datasets

In [ ]:
DATA_DIR = '/content/drive/MyDrive/DATASET/'

Set the argumentation of hyperparameters

In [ ]:
# Framework:
# 'centralized' -----Centralized Semantic Segmentation learning
# 'Federated' -----Federated Semantic Segmentation learning

# DATASET Partition: A/B
# Clients type: uniform/heterogeneous

# When set the framework to 'federated' the clients_type should be specified correspondingly
# !!!set random_seed to None if it needs to drop the DDP

import dataset
from utils import parse_args, modify_command_options
from run import run_experiment

parser = parse_args()
args = parser.parse_args(['--framework', 'federated', '--dataset', 'cityscapes', '--partition', 'A'])

args.device_ids = [0]
args.random_seed = 0
args.wandb_entity = 'feddrive2023'
args.mixed_precision = True
args.ignore_warnings = True

args.save_samples = True

args.num_rounds = 1
args.clients_per_round = 1
args.num_epochs = 30

args.hnm = True
args.output_aux = True
args.batch_size = 16
args.eval_interval = 1
args.test_batch_size = 1

args.server_opt = 'SGD'
args.weight_decay = 0.0005
args.custom_lr_param = True

args.lr = 0.15 #0.007, 0.01, 0.015, 0.02, 0.025, 0.05, 0.1, 0.125, 0.15, 0.2
args.lr_policy = 'poly'
args.lr_power = 0.9

args.jitter = True
args.rrc_transform = True
args.min_scale = 0.5
args.max_scale = 1.5
args.h_resize = 512
args.w_resize = 1024

if args.framework == 'federated':
  args.clients_type = 'uniform'
  args.double_dataset = True
  args.name = f'cityscapes_{args.clients_type}'
  args.avg_last_100 = True
  args.num_rounds = 60
  args.clients_per_round = 5
  args.num_epochs = 3
  args.server_opt = 'SGD'
  args.weight_decay = 0.0005
  args.momentum = 0.9
  args.server_opt = 'SGD'
  args.server_lr = 1
  args.server_momentum = 0

args = modify_command_options(args)

print(args)

Execute the experiment

In [ ]:
if args.ignore_warnings:
  warnings.filterwarnings("ignore")

start = time.time()

run_experiment(args)

end = time.time()
print(f"Elapsed time: {round(end - start, 2)}")